In [12]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.occ import *
from netgen.csg import *


In [13]:
E, nu = 50, 0.1
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(3)


In [14]:
L = 1
w = 0.2
left = Plane(Pnt(0,0,0), Vec(-1,0,0)).bc("left")
right = Plane(Pnt(L,0,0), Vec(1,0,0)).bc("right")
bottom = Plane(Pnt(0,0,0), Vec(0,-1,0)).bc("bottom")
top = Plane(Pnt(0,w,0), Vec(0,1,0)).bc("top")
front = Plane(Pnt(0,0,0), Vec(0,0,-1)).bc("front")
back = Plane(Pnt(0,0,w), Vec(0,0,1)).bc("back")
cube = left * right * bottom * top * front * back

geo = CSGeometry()
geo.Add(cube)
mesh = Mesh(geo.GenerateMesh(maxh=0.1))
Draw(mesh)


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [ ]:
# H1-conforming finite element space
fesu = VectorH1(mesh, order=2, dirichlet='left')
fesphi = L2(mesh, order=0)

fes = FESpace([fesu,fesphi])
# define trial- and test-functions
u, phi = fes.TrialFunction()
v, vphi = fes.TestFunction()
gfu_u = GridFunction(fesu)
gfu_u.Set((0,0,1))

psih = GridFunction(fesphi)
psih.Set(1)

uk = GridFunction(fesu)
psik = GridFunction(fesphi)
eps = 1e-6
# define phi as the 0 function 
obs = CF(-0.1)    

max_PG_it = 20
max_iter_newton = 30
rho = 0.1
grav = 9.81
force = CoefficientFunction((0,0,-rho*grav))
alpha = Parameter(1)
gfu = GridFunction(fes)
for k in range(1,max_PG_it):

    alpha.Set(2**k)
    print("Iteration of proximal galerkin: ", k)
    # assing uk, psik
    uk.vec.data = gfu_u.vec
    psik.vec.data = psih.vec
    a = BilinearForm(fes)

    with TaskManager():
        # the stress tensor equation
        a += alpha * InnerProduct(Stress(Sym(Grad(u))), Sym(Grad(v))).Compile()*dx
        # proximal galerkin term
        a+= InnerProduct(phi,v[2])*dx
        a+= InnerProduct(u[2], vphi)*dx
        a+= -InnerProduct(exp(phi),vphi)*dx
        # the right hand side

        a += -alpha * force[2] * v[2] * dx 
        a += -(psik)*v[2]*dx
        a += -(obs)*vphi*dx
    # solve non linear problem
    
    # the solution field 
    
    res = gfu.vec.CreateVector()
    w = gfu.vec.CreateVector()
    for it in range(max_iter_newton):
        a.Apply(gfu.vec, res)
        a.AssembleLinearization(gfu.vec)
        inv = a.mat.Inverse(freedofs=fes.FreeDofs())
        w.data = 0.3*inv * res
        gfu.vec.data -= w
        if abs(InnerProduct(w,w)) < 1e-8:
            print("Tol newton")
            break
    Draw(gfu.components[0], mesh, "u")
    # r.data = lf.vec - a.mat * gfu.vec
    #gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * lf.vec
    if Integrate((gfu_u-gfu.components[0])**2, mesh) < eps:
        print("Tol PG")
        break
    gfu_u.vec.data = gfu.components[0].vec
    psih.vec.data = psih.vec + gfu.components[1].vec



Iteration of proximal galerkin:  1
Newton iteration  0
Newton iteration  1
Newton iteration  2
Newton iteration  3
Newton iteration  4
Newton iteration  5
Newton iteration  6
Newton iteration  7
Newton iteration  8
Newton iteration  9
Newton iteration  10
Newton iteration  11
Newton iteration  12
Newton iteration  13
Newton iteration  14
Newton iteration  15
Newton iteration  16
Newton iteration  17
Newton iteration  18
Newton iteration  19
Newton iteration  20
Newton iteration  21
Newton iteration  22
Newton iteration  23
Newton iteration  24
Newton iteration  25
Newton iteration  26
Newton iteration  27
Newton iteration  28
Newton iteration  29


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Iteration of proximal galerkin:  2
Newton iteration  0
Newton iteration  1
Newton iteration  2
Newton iteration  3
Newton iteration  4
Newton iteration  5
Newton iteration  6
Newton iteration  7
Newton iteration  8
Newton iteration  9
Newton iteration  10
Newton iteration  11
Newton iteration  12
Newton iteration  13
Newton iteration  14
Newton iteration  15
Newton iteration  16
Newton iteration  17
Newton iteration  18
Newton iteration  19
Newton iteration  20
Newton iteration  21
Newton iteration  22
Newton iteration  23
Newton iteration  24
Newton iteration  25
Newton iteration  26
Newton iteration  27
Newton iteration  28
Newton iteration  29


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Iteration of proximal galerkin:  3
Newton iteration  0
Newton iteration  1
Newton iteration  2
Newton iteration  3
Newton iteration  4
Newton iteration  5
Newton iteration  6
Newton iteration  7
Newton iteration  8
Newton iteration  9
Newton iteration  10
Newton iteration  11
Newton iteration  12
Newton iteration  13
Newton iteration  14
Newton iteration  15
Newton iteration  16
Newton iteration  17
Newton iteration  18
Newton iteration  19
Newton iteration  20
Newton iteration  21
Newton iteration  22
Newton iteration  23
Newton iteration  24
Newton iteration  25
Newton iteration  26
Newton iteration  27
Newton iteration  28
Newton iteration  29


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Iteration of proximal galerkin:  4
Newton iteration  0
Newton iteration  1
Newton iteration  2
Newton iteration  3
Newton iteration  4
Newton iteration  5
Newton iteration  6
Newton iteration  7
Newton iteration  8
Newton iteration  9
Newton iteration  10
Newton iteration  11
Newton iteration  12
Newton iteration  13
Newton iteration  14
Newton iteration  15
Newton iteration  16
Newton iteration  17
Newton iteration  18
Newton iteration  19
Newton iteration  20
Newton iteration  21
Newton iteration  22
Newton iteration  23
Newton iteration  24
Newton iteration  25
Newton iteration  26
Newton iteration  27
Newton iteration  28
Newton iteration  29


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Tol PG


In [17]:
import pickle
with open('gfu.pkl', 'wb') as output:
    pickle.dump(gfu_u, output)
